In [1]:
import sys
sys.path.append('../../build')
import libry as ry
from ipywidgets import interact
import numpy as np
import time


In [2]:
C = ry.Config()
C.addFile('../../../scenarios/pandasTable.g')
V = ry.ConfigurationViewer()
cameraFrame = C.frame("camera")

q0 = C.getJointState()
R_gripper = C.frame("R_gripper")
R_gripper.setContact(1)
L_gripper = C.frame("L_gripper")
L_gripper.setContact(1)

V.setConfiguration(C)

In [3]:
#set  flat object on top of a table on the table edge
tab = C.addFrame("table2")
tab.setShape(ry.ST.ssBox, [0.5, 0.5, 0.5, 0])
#tab.setColor([0,0,1])
tab.setMass(0.2)
tab.setPosition([.0, .3, .9])

flip = C.addFrame("flipObject")
flip.setColor([0, 0, 1])
flip.setShape(ry.ST.ssBox, [0.4, 0.55, 0.01, 0.01])
flip.setPosition([0, .1, 1.15])
flip.setMass(10)
flip.setContact(1)

#@interact(x=(-1, 1, 0.1), y = (-1, 1, 0.1), z = (0, 2, 0.1), a = (0, 1, 0.1))
#def set_position(x, y, z):
#    flip.setPosition([x, y, z])
V.setConfiguration(C)

X0 = C.getFrameState()

In [4]:
C.setFrameState(X0)
IK = C.komo_IK(False)
IK.addObjective(type=ry.OT.eq, feature=ry.FS.scalarProductXX, frames=['flipObject', 'R_gripperCenter'], target = [1])
IK.addObjective(type=ry.OT.eq, feature=ry.FS.scalarProductZZ, frames=['flipObject', 'R_gripperCenter'], target = [0])
IK.addObjective(type=ry.OT.eq, feature=ry.FS.positionRel, frames=['flipObject','R_gripperCenter'], target=[0 , .2, -.2])
IK.optimize()
C.setFrameState( IK.getConfiguration(0) )
q_below = C.getJointState()

In [5]:
C.setFrameState(X0)
IK = C.komo_IK(False)
IK.addObjective(type=ry.OT.eq, feature=ry.FS.scalarProductXX, frames=['flipObject', 'R_gripperCenter'], target = [1])
IK.addObjective(type=ry.OT.eq, feature=ry.FS.scalarProductZZ, frames=['flipObject', 'R_gripperCenter'], target = [0])
IK.addObjective(type=ry.OT.eq, feature=ry.FS.positionRel, frames=['flipObject','R_gripperCenter'], target=[0 , -.4, -.2])
IK.optimize()
C.setFrameState( IK.getConfiguration(0) )
q_above = C.getJointState()

In [6]:
S = C.simulation(ry.SimulatorEngine.physx, True)
S.addSensor("camera")

In [ ]:
T = 10
tau = .01

#flip the object - or try it :D
C.setJointState(q_below)

komo = C.komo_path(1, T, T * tau, True)
komo.addObjective(time=[1.], feature=ry.FS.qItself, type=ry.OT.eq, target=q_above, scale=[1e2], order=0)
komo.addObjective(time=[1.], feature=ry.FS.qItself, type=ry.OT.eq, scale=[1e2], order=1)
komo.optimize()
komo.getReport()
t = 0
time.sleep(3)
while True:
    t += 1
    if t < T:
        C.setFrameState(komo.getConfiguration(t))
    else:
        C.setJointState(q_above)
    q = C.getJointState()
    S.step(q, tau, ry.ControlMode.position)
    time.sleep(0.2)



